# Import libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, r2_score, f1_score, confusion_matrix, roc_curve
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn import metrics

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve
import plotly.graph_objects as go
     

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

# Read file with labels

In [3]:
data = pd.read_csv('conversion_data_train.csv')
data_test = pd.read_csv('conversion_data_test.csv')

In [ ]:
print("valeurs manquantes des datasets train et test: ")
print(data.isnull().sum())
print('-----')
print(data_test.isnull().sum())

# Explore dataset

In [ ]:
print('statistiques basiques')
print(data.describe())

In [ ]:
print(data.groupby(['country']).size())
print('----')
print(data.groupby(['source']).size())

In [ ]:
print("données aberrantes")
data[data['age'] >= 100]

In [ ]:
# Suppression des données aberrantes 
data = data[data['age'] < 100]

In [ ]:
print("Le dataset est trop volumineux piur l EDA : on réduit le dataser à 10000 lignes pour l'EDA.")
data_sample = data.sample(10000)

In [ ]:
# correlation
cor = data
del cor['converted']

plt.figure(figsize=(7,5))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Visualizing the 'age' distribution
sns.distplot(data_sample['age'])

In [ ]:
print("la Chine compte un plus grand nombre de visiteurs sur le site.")
print("Le taux de conversion est beaucoup plus faible que les autres pays")

print('')
data_sample.groupby(['country'])['converted'].mean().plot.bar()
plt.show()

In [ ]:
print("User s'abonnant selon le canal :")

sns.barplot(x='source',y='converted',data=data_sample)

In [ ]:
print("plus de temps sur le site Web entraîne plus de conversions à la news letter :")
print('')
data_sample.groupby(['total_pages_visited'])['converted'].mean().plot()

In [ ]:
print("trop peu de pages visitées :")
sns.countplot(x='total_pages_visited', data=data_sample)

# modelisation

###  Modéle basé sur la variable : total_pages_visited (algorithme de classification)

In [ ]:
data = pd.read_csv('conversion_data_train.csv')

In [ ]:
target_variable = ['converted']
features_list = ['total_pages_visited']

In [ ]:
X = data.loc[:, features_list]
Y = data.loc[:, target_variable]

In [ ]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)


In [ ]:
# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values


## Training pipeline

In [ ]:
print("Encoding categorical features and standardizing numerical features...")

featureencoder = StandardScaler()
X_train = featureencoder.fit_transform(X_train)

# instancier le Train model 
print("Train model...")
classifier = LogisticRegression() 
classifier.fit(X_train, Y_train)

# Predictions on training set
print("Predictions on training set...")
Y_train_pred = classifier.predict(X_train)


## Test pipeline

In [ ]:
print("Encoding categorical features and standardizing numerical features...")

X_test = featureencoder.transform(X_test)


In [ ]:
print("Predictions on test set...")
Y_test_pred = classifier.predict(X_test)



## Performance assessment

In [ ]:
print("Confusion matrix on train set : ")
print(confusion_matrix(Y_train, Y_train_pred))
print()
print("Confusion matrix on test set : ")
print(confusion_matrix(Y_test, Y_test_pred))


In [ ]:
print(classification_report(Y_test, Y_test_pred))

In [ ]:
print("f1-score on train set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))

In [ ]:
print("Accuracy:",metrics.accuracy_score(Y_test, Y_test_pred))
print("Precision:",metrics.precision_score(Y_test, Y_test_pred))
print("Recall:",metrics.recall_score(Y_test, Y_test_pred))

**Our baseline model reaches a f1-score of almost 70%. Now, feel free to refine your model and try to beat this score ! 🚀🚀**

###  Modèle basé sur toutes les variables 

In [ ]:
data_all = pd.read_csv('conversion_data_train.csv')

In [ ]:
data_final = pd.get_dummies(data)

In [ ]:
# on sépare la target des autres variables

features_list = ['age','new_user','total_pages_visited','country_China','country_Germany','country_UK','country_US','source_Ads',
                 'source_Direct','source_Seo']
target_variable = 'converted'

X = data_final.loc[:, features_list]
Y = data_final.loc[:, target_variable]


In [ ]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
print()

# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values


#### Training pipeline  with all variables : Preprocessings - Train model - Predictions on training set

In [ ]:
# Preprocessings
print("Encoding categorical features and standardizing numerical features...")

featureencoder = StandardScaler()
X_train = featureencoder.fit_transform(X_train)
print("")


In [ ]:
print("Train model...")
classifier = LogisticRegression() 
classifier.fit(X_train, Y_train)
print("")


In [ ]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = classifier.predict(X_train)
print("")

#### Test pipeline  with all variables : Preprocessings - Predictions on test set

In [ ]:
print("Encoding categorical features and standardizing numerical features...")

X_test = featureencoder.transform(X_test)

print("Predictions on test set...")
Y_test_pred = classifier.predict(X_test)
print("")

In [ ]:
print("Confusion matrix on train set : ")
print(confusion_matrix(Y_train, Y_train_pred))
print()
print("Confusion matrix on test set : ")
print(confusion_matrix(Y_test, Y_test_pred))
print()


In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'
      .format(classifier.score(X_test, Y_test)))

In [ ]:
print(classification_report(Y_test, Y_test_pred))

In [ ]:

print("f1-score on train set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))

In [ ]:
# Visualize ROC curves


**Our baseline model reaches a f1-score of almost 76% ... try other models DecisionTree & Random Forest 🚀🚀**

### Decision Tree and Random Forest


In [31]:
dataset  = data.copy()

### Preprocessing

#### Training pipeline

In [32]:
print("Separating labels from features...")
features_list = ["country", "age", "new_user","source","total_pages_visited"]
target_variable = "converted"

X = dataset.loc[:,features_list]
Y = dataset.loc[:,target_variable]

Separating labels from features...


In [33]:
# Automatically detect positions of numeric/categorical features
idx = 0
numeric_features = []
numeric_indices = []
categorical_features = []
categorical_indices = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
        numeric_indices.append(idx)
    else :
        categorical_features.append(i)
        categorical_indices.append(idx)

    idx = idx + 1

print('Found numeric features ', numeric_features,' at positions ', numeric_indices)
print('Found categorical features ', categorical_features,' at positions ', categorical_indices)

Found numeric features  ['age', 'new_user', 'total_pages_visited']  at positions  [1, 2, 4]
Found categorical features  ['country', 'source']  at positions  [0, 3]


In [34]:
print("Dividing into train and test sets...")

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify = Y)


Dividing into train and test sets...


In [35]:
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.tolist()
Y_test = Y_test.tolist()

Convert pandas DataFrames to numpy arrays...


#### Training pipeline

In [36]:
# Missing values
print("Imputing missing values...")

imputer = SimpleImputer(strategy="mean")
X_train[:,numeric_indices] = imputer.fit_transform(X_train[:,numeric_indices])

# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print()

# Normalization
numeric_transformer = StandardScaler()

# OHE / dummyfication
categorical_transformer = OneHotEncoder(drop='first')

featureencoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_indices),    
        ('num', numeric_transformer, numeric_indices)
        ]
    )

X_train = featureencoder.fit_transform(X_train)


# Label encoding
print("Encoding labels...")
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)


Imputing missing values...
Encoding categorical features and standardizing numerical features...

Encoding labels...


#### Test pipeline

In [37]:
# Missing values
print("Imputing missing values...")

X_test[:,numeric_indices] = imputer.transform(X_test[:,numeric_indices])

# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
X_test = featureencoder.transform(X_test)


# Label encoding
print("Encoding labels...")
Y_test = encoder.transform(Y_test)

Imputing missing values...
Encoding categorical features and standardizing numerical features...
Encoding labels...


##### Decision Tree

In [38]:
# DecisionTree

# Perform grid search
print("Grid search...")
classifier = DecisionTreeClassifier()

# Grid of values to be tested
params = {
    'max_depth': [4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8]
}
gridsearch = GridSearchCV(classifier, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best validation accuracy :  0.9855137395459975


In [39]:
#Performances

# Predictions on training set
Y_train_pred = gridsearch.predict(X_train)
print(Y_train_pred)

[0 0 0 ... 0 0 0]


In [13]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = gridsearch.predict(X_test)

print(Y_test_pred)
print()

Predictions on test set...
[0 0 0 ... 0 0 0]



In [40]:
# Print scores
print("accuracy on training set : ", accuracy_score(Y_train, Y_train_pred))
print("accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))
print()

print("f1-score on training set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))
print()

accuracy on training set :  0.9862824161922834
accuracy on test set :  0.9846440368262

f1-score on training set :  0.7635702929820577
f1-score on test set :  0.735632183908046



In [41]:
# Visualize ROC curves
probas_train = gridsearch.predict_proba(X_train)[:,1]
precisions, recalls, thresholds = roc_curve(Y_train, probas_train)
fig = go.Figure(
    data = go.Scatter(
        name = 'train',
        x = recalls, 
        y = precisions, 
        mode = 'lines'
    ),
    layout = go.Layout(
        title = go.layout.Title(text = "ROC curve", x = 0.5),
        xaxis = go.layout.XAxis(title = 'False Positive Rate'),
        yaxis = go.layout.YAxis(title = 'True Positive Rate')
    )
)

probas_test = gridsearch.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = roc_curve(Y_test, probas_test)
fig.add_trace(go.Scatter(
    name = 'test',
    x = recalls, 
    y = precisions, 
    mode = 'lines'
    )
)
fig.show()

##### Random Forest 

In [44]:
rf_classifier = RandomForestClassifier(n_estimators=50)
rf_classifier.fit(X_train, Y_train)
rf_classifier.score(X_train, Y_train)

print("score avec 50 estimators")
rf_classifier.score(X_test, Y_test)

score avec 50 estimators


0.9840466652610865

score avec 100 estimators


0.9842399325321527

In [22]:
# Make a grid search to find the optimal number of trees

from sklearn.model_selection import GridSearchCV
rf_classifier_grid = RandomForestClassifier()
params = {"n_estimators": range(20, 50)}

classifier = GridSearchCV(cv=3, estimator = rf_classifier_grid, param_grid = params).fit(X_train, Y_train)

In [46]:
classifier.best_params_

AttributeError: 'DecisionTreeClassifier' object has no attribute 'best_params_'

In [25]:
classifier.best_estimator_.score(X_test, Y_test)

0.9840993745168318

In [27]:
# Analyze the random forest's feature importances
feature_importance = pd.DataFrame({"features": dataset.columns[:-1], 
                                   "score": classifier.best_estimator_.feature_importances_}).sort_values(by="score")
feature_importance


ValueError: arrays must all be same length

In [28]:
param = {"n_estimators": np.arange(40,50),
        "max_depth": np.arange(2,6),
        "min_samples_split": np.arange(2,5)}

In [30]:
rf = RandomForestClassifier()
grid = GridSearchCV(rf, param_grid=param)
grid.fit(X_train,Y_train)

KeyboardInterrupt: 

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_.score(X_train, Y_train)

In [ ]:
grid.best_estimator_.score(X_test, Y_test)